In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 2022

In [2]:
team_stats = pd.read_csv('../../footy_data/england-efl-league-one-teams-2021-to-2022-stats.csv')
games = pd.read_csv('../../footy_data/england-efl-league-one-matches-2021-to-2022-stats.csv')

In [3]:
games = games.loc[games['status'] == 'complete']
games
#Only include completed games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bolton Wanderers,Milton Keynes Dons,Martin Coy,1,0.00,0.00,...,2.45,3.35,2.75,1.29,1.80,2.80,4.95,1.69,2.15,University of Bolton Stadium (Bolton)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cambridge United,Oxford United,David Rock,1,0.00,0.00,...,3.55,3.40,2.05,1.35,1.95,3.10,5.50,1.80,2.00,Abbey Stadium (Cambridge- Cambridgeshire)
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Crewe Alexandra,Cheltenham Town,Ben Toner,1,0.00,0.00,...,2.25,3.05,3.40,1.39,2.10,3.40,6.75,1.87,1.91,Mornflake Stadium (Crewe- Cheshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Doncaster Rovers,AFC Wimbledon,Samuel Barrott,1,0.00,0.00,...,2.40,3.25,2.95,1.34,1.95,3.10,5.75,1.77,2.00,The Keepmoat Stadium (Doncaster- South Yorkshire)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Fleetwood Town,Portsmouth,Ben Speedie,1,0.00,0.00,...,2.95,3.15,2.40,1.48,2.25,3.90,7.50,2.05,1.74,Highbury Stadium (Fleetwood- Lancashire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,1634997600,Oct 23 2021 - 2:00pm,complete,NaN,Shrewsbury Town,Cambridge United,Simon Mather,15,1.43,1.25,...,2.29,3.20,3.10,1.36,2.00,3.88,7.50,1.80,1.95,Montgomery Waters Meadow (Shrewsbury- Shropshire)
167,1634997600,Oct 23 2021 - 2:00pm,complete,NaN,Sunderland,Charlton Athletic,Darren Drysdale,15,3.00,0.67,...,1.64,3.85,4.90,1.27,1.88,3.20,6.06,1.80,1.95,Stadium of Light (Sunderland)
168,1634997600,Oct 23 2021 - 2:00pm,complete,NaN,Wycombe Wanderers,Crewe Alexandra,Steve Martin,15,3.00,0.43,...,1.49,4.30,6.00,1.26,1.88,3.10,5.90,1.95,1.80,Adams Park (High Wycombe- Buckinghamshire)
169,1635273900,Oct 26 2021 - 6:45pm,complete,NaN,Doncaster Rovers,Cambridge United,Andy Haines,6,1.67,1.00,...,2.80,3.20,2.55,1.35,2.10,3.55,6.85,1.78,1.95,The Keepmoat Stadium (Doncaster- South Yorkshire)


Seeing how many games had goals and xg of over 2.5

In [4]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [5]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100
percent_goals_1h

44.349680170575695

In [6]:
games['over/under'].value_counts()

1    90
0    80
Name: over/under, dtype: int64

In [7]:
games['xg_over/under'].value_counts()

0    99
1    71
Name: xg_over/under, dtype: int64

In [8]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]

In [11]:
xg_view[120:180]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
120,Crewe Alexandra,Cambridge United,1.13,1.83,2,2
121,Doncaster Rovers,Milton Keynes Dons,0.85,1.40,2,1
122,Fleetwood Town,Charlton Athletic,1.57,1.63,1,2
123,Gillingham,Wigan Athletic,1.34,1.81,0,2
124,Lincoln City,Plymouth Argyle,1.28,1.61,2,2
125,Portsmouth,Sunderland,1.73,1.29,4,0
126,Sheffield Wednesday,Oxford United,1.81,1.78,1,2
127,Wycombe Wanderers,Morecambe,2.18,1.75,4,3
128,Ipswich Town,Shrewsbury Town,1.28,0.67,2,1
129,Plymouth Argyle,Burton Albion,1.95,1.14,2,1


In [10]:
first_matchweek = games[0:12]

In [11]:
first_matchweek['over/under'].value_counts()

0    7
1    5
Name: over/under, dtype: int64

In [13]:
first_matchweek['over/under_1.5'].value_counts()

1    9
0    3
Name: over/under_1.5, dtype: int64

In [14]:
games['over/under_1.5'].value_counts()

1    120
0     36
Name: over/under_1.5, dtype: int64

In [15]:
games['over/under'].value_counts()

1    81
0    75
Name: over/under, dtype: int64

In [16]:
games['total_goals_at_half_time'].value_counts()

1    52
2    48
0    42
3    11
4     2
5     1
Name: total_goals_at_half_time, dtype: int64

In [20]:
games['home_team_goal_count_half_time'].sum()

75

In [21]:
games['away_team_goal_count_half_time'].sum()

67

# Teams

In [22]:
team_stats[['team_name', 'xg_for_avg_overall', 'xg_against_avg_overall']]

,team_name,xg_for_avg_overall,xg_against_avg_overall
0,Sunderland AFC,1.64,1.29
1,Burton Albion FC,1.60,1.39
2,Rotherham United FC,1.79,0.83
3,Sheffield Wednesday FC,0.81,1.11
4,Ipswich Town FC,1.16,1.00
5,Wigan Athletic FC,1.21,1.04
6,Charlton Athletic FC,1.19,1.31
7,Bolton Wanderers FC,1.60,1.20
8,Milton Keynes Dons FC,1.83,1.03
9,Doncaster Rovers FC,0.62,1.46


sos = total average of xg for xg against of opponents they have played

# Adding More Columns

In [23]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

In [24]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats

,common_name,xg_against_avg_overall
0,Sunderland,1.29
1,Burton Albion,1.39
2,Rotherham United,0.83
3,Sheffield Wednesday,1.11
4,Ipswich Town,1.00
5,Wigan Athletic,1.04
6,Charlton Athletic,1.31
7,Bolton Wanderers,1.20
8,Milton Keynes Dons,1.03
9,Doncaster Rovers,1.46


In [25]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [26]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [27]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 